Module 1 Homework
In this homework, we're going to download finance data from various sources and make simple calculations/analysis.

Question 1: [Macro] Average growth of GDP in 2023
What is the average growth (in %) of GDP in 2023?

Download the timeseries Real Gross Domestic Product (GDPC1) from FRED (https://fred.stlouisfed.org/series/GDPC1). Calculate year-over-year (YoY) growth rate (that is, divide current value to one 4 quarters ago). Find the average YoY growth in 2023 (average from 4 YoY numbers). Round to 1 digit after the decimal point: e.g. if you get 5.66% growth => you should answer 5.7

In [7]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

In [8]:
end = date.today()
print(f'Year = {end.year}; month= {end.month}; day={end.day}')

start = date(year=end.year-5, month=end.month, day=end.day)
print(f'Period for indexes: {start} to {end} ')

Year = 2024; month= 4; day=21
Period for indexes: 2019-04-21 to 2024-04-21 


In [19]:
gdppot = pdr.DataReader("GDPC1", "fred", start=start)

In [20]:
gdppot['YoY_Growth'] = gdppot.GDPC1/gdppot.GDPC1.shift(4)-1
gdppot['% YoY_Growth'] = round(gdppot['YoY_Growth'] * 100, 1)
gdppot.tail()

,GDPC1,YoY_Growth,% YoY_Growth
DATE,,,
2022-10-01,21989.981,0.006517,0.7
2023-01-01,22112.329,0.017179,1.7
2023-04-01,22225.350,0.023825,2.4
2023-07-01,22490.692,0.029269,2.9
2023-10-01,22679.255,0.031345,3.1


In [29]:
avg_growth = gdppot[gdppot.index.year == 2023]
avg_growth_avg = avg_growth['% YoY_Growth'].mean()

In [30]:

print(f"The average YoY GDP growth rate in 2023 was {avg_growth_avg:.1f}%")

The average YoY GDP growth rate in 2023 was 2.5%


### Question 2. \[Macro\] Inverse "Treasury Yield"

[](https://github.com/DataTalksClub/stock-markets-analytics-zoomcamp/blob/main/cohorts/2024/homework%201.md#question-2-macro-inverse-treasury-yield)

**Find the min value of (dgs10-dgs2) after since year 2000 (2000-01-01) and write it down as an answer, round to 1 digit after the decimal point.**

Download DGS2 and DGS10 interest rates series ([https://fred.stlouisfed.org/series/DGS2](https://fred.stlouisfed.org/series/DGS2), [https://fred.stlouisfed.org/series/DGS10](https://fred.stlouisfed.org/series/DGS10)). Join them together to one dataframe on date (you might need to read about pandas.DataFrame.join()), calculate the difference dgs10-dgs2 daily.

(Additional: think about what does the "inverted yield curve" mean for the market and investors? do you see the same thing in your country/market of interest? Do you think it can be a good predictive feature for the models?)

In [31]:
dgs2 = pdr.DataReader("DGS2", "fred", start=date(2000, 1, 1))
dgs2.tail()

,DGS2
DATE,
2024-04-12,4.88
2024-04-15,4.93
2024-04-16,4.97
2024-04-17,4.93
2024-04-18,4.98


In [32]:
dgs10 = pdr.DataReader("DGS10", "fred", start=date(2000, 1, 1))
dgs10.tail()

,DGS10
DATE,
2024-04-12,4.50
2024-04-15,4.63
2024-04-16,4.67
2024-04-17,4.59
2024-04-18,4.64


In [38]:
df_dgs210 = dgs2.join(dgs10, on='DATE', how='inner')

In [40]:
df_dgs210.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6339 entries, 2000-01-03 to 2024-04-18
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   DGS2    6078 non-null   float64
 1   DGS10   6078 non-null   float64
dtypes: float64(2)
memory usage: 148.6 KB


In [42]:
# Fill NAN
df_dgs210.replace('.', np.nan, inplace=True)
df_dgs210['DGS2'] = pd.to_numeric(df_dgs210['DGS2'], errors='coerce')
df_dgs210['DGS10'] = pd.to_numeric(df_dgs210['DGS10'], errors='coerce')

In [43]:
df_dgs210.tail()

,DGS2,DGS10
DATE,,
2024-04-12,4.88,4.50
2024-04-15,4.93,4.63
2024-04-16,4.97,4.67
2024-04-17,4.93,4.59
2024-04-18,4.98,4.64


In [44]:
df_dgs210.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6339 entries, 2000-01-03 to 2024-04-18
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   DGS2    6078 non-null   float64
 1   DGS10   6078 non-null   float64
dtypes: float64(2)
memory usage: 148.6 KB


In [45]:
# Calculate the difference DGS10 - DGS2
df_dgs210['Diff'] = df_dgs210.DGS10 - df_dgs210.DGS2

In [46]:
# Find the minimum value of the difference
min_diff = df_dgs210['Diff'].min()

print(f'The minimum is: {min_diff:.1f}')

The minimum is: -1.1


### Question 3. \[Index\] Which Index is better recently?

[](https://github.com/DataTalksClub/stock-markets-analytics-zoomcamp/blob/main/cohorts/2024/homework%201.md#question-3-index-which-index-is-better-recently)

**Compare S&P 500 and IPC Mexico indexes by the 5 year growth and write down the largest value as an answer (%)**

Download on Yahoo Finance two daily index prices for S&P 500 (^GSPC, [https://finance.yahoo.com/quote/%5EGSPC/](https://finance.yahoo.com/quote/%5EGSPC/)) and IPC Mexico (^MXX, [https://finance.yahoo.com/quote/%5EMXX/](https://finance.yahoo.com/quote/%5EMXX/)). Compare 5Y growth for both (between 2019-04-09 and 2024-04-09). Select the higher growing index and write down the growth in % (closest integer %). E.g. if ratio end/start was 2.0925 (or growth of 109.25%), you need to write down 109 as your answer.

(Additional: think of other indexes and try to download stats and compare the growth? Do create 10Y and 20Y growth stats. What is an average yearly growth rate (CAGR) for each of the indexes you select?)

In [61]:
start_date = '2019-04-09'
end_date = '2024-04-09'

In [62]:
# INDEXES from Yahoo Finance
# WEB: https://finance.yahoo.com/quote/%5EGSPC/
sp500_daily = yf.download(tickers = "^GSPC",
                     period = "max",
                     interval = "1d")

[*********************100%%**********************]  1 of 1 completed


In [63]:
sp500_daily.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-04-15,5149.669922,5168.430176,5052.470215,5061.819824,5061.819824,3950210000
2024-04-16,5064.589844,5079.839844,5039.830078,5051.410156,5051.410156,4006200000
2024-04-17,5068.970215,5077.959961,5007.250000,5022.209961,5022.209961,3596130000
2024-04-18,5031.520020,5056.660156,5001.890137,5011.120117,5011.120117,3619760000
2024-04-19,5005.439941,5019.020020,4953.560059,4967.229980,4967.229980,3878750000


In [64]:
# Extract close prices

sp500_start = sp500_daily.loc[start_date, 'Close']
sp500_end = sp500_daily.loc[end_date, 'Close']

In [65]:
# INDEXES from Yahoo Finance
# WEB: https://finance.yahoo.com/quote/%5EGSPC/
mxx_daily = yf.download(tickers = "^MXX",
    period = "max",
    interval = "1d")

[*********************100%%**********************]  1 of 1 completed


In [66]:
mxx_daily.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-04-15,56536.468750,56709.121094,55924.101562,55984.000000,55984.000000,170645700
2024-04-16,56009.730469,56148.000000,55626.750000,55797.250000,55797.250000,225862900
2024-04-17,55788.738281,56090.320312,55383.250000,55415.691406,55415.691406,244258900
2024-04-18,55597.789062,55941.621094,55585.820312,55739.429688,55739.429688,208671900
2024-04-19,55811.191406,56036.718750,55579.691406,55862.851562,55862.851562,138617600


In [67]:
mxx_start = mxx_daily.loc[start_date, 'Close']
mxx_end = mxx_daily.loc[end_date, 'Close']

In [71]:
# Calculate growth percentages
sp500_growth = ((sp500_end - sp500_start) / sp500_start) * 100
mxx_growth = ((mxx_start - mxx_end) / mxx_start) * 100

In [72]:
# Determine the higher growth and print it
higher_growth = max(sp500_growth, mxx_growth)
print(f"The higher 5-year growth between S&P 500 and IPC Mexico is approximately {int(higher_growth)}%.")

The higher 5-year growth between S&P 500 and IPC Mexico is approximately 81%.


### Question 4. \[Stocks OHLCV\] 52-weeks range ratio (2023) for the selected stocks

[](https://github.com/DataTalksClub/stock-markets-analytics-zoomcamp/blob/main/cohorts/2024/homework%201.md#question-4-stocks-ohlcv-52-weeks-range-ratio-2023-for-the-selected-stocks)

**Find the largest range ratio \[=(max-min)/max\] of Adj.Close prices in 2023**

Download the 2023 daily OHLCV data on Yahoo Finance for top6 stocks on earnings ([https://companiesmarketcap.com/most-profitable-companies/](https://companiesmarketcap.com/most-profitable-companies/)): 2222.SR,BRK-B, AAPL, MSFT, GOOG, JPM.

Here is the example data you should see in Pandas for "2222.SR": [https://finance.yahoo.com/quote/2222.SR/history](https://finance.yahoo.com/quote/2222.SR/history)

Calculate maximum-minimim "Adj.Close" price for each stock and divide it by the maximum "Adj.Close" value. Round the result to two decimal places (e.g. 0.1575 will be 0.16)

(Additional: why this may be important for your research?)

In [77]:
start_date = '2023-01-01'
end_date = '2023-12-31'
ticker_list = ['2222.SR', 'BRK-B', 'AAPL', 'MSFT', 'GOOG', 'JPM']

ohlcv_2023_daily = yf.download(ticker_list,
    start=start_date,
    end=end_date,
    period = "max",
    interval = "1d")

[****************      33%%                      ]  2 of 6 completed

[*********************100%%**********************]  6 of 6 completed


In [84]:
ohlcv_2023_daily.head()

Price       Adj Close                                                 \
Ticker        2222.SR        AAPL       BRK-B       GOOG         JPM   
Date                                                                   
2023-01-01  28.475794         NaN         NaN        NaN         NaN   
2023-01-02  28.256075         NaN         NaN        NaN         NaN   
2023-01-03  28.256075  124.216301  309.910004  89.699997  129.648499   
2023-01-04  27.860575  125.497505  314.549988  88.709999  130.857483   
2023-01-05  27.728743  124.166641  312.899994  86.769997  130.828476   

Price                       Close                                     ...  \
Ticker            MSFT    2222.SR        AAPL       BRK-B       GOOG  ...   
Date                                                                  ...   
2023-01-01         NaN  29.454544         NaN         NaN        NaN  ...   
2023-01-02         NaN  29.227272         NaN         NaN        NaN  ...   
2023-01-03  237.036011  29.227272  125.070000  309.910004  89.699997  ...   
2023-01-04  226.667282  28.818180  126.360001  314.549988  88.709999  ...   
2023-01-05  219.949371  28.681818  125.019997  312.899994  86.769997  ...   

Price             Open                                        Volume  \
Ticker           BRK-B       GOOG         JPM        MSFT    2222.SR   
Date                                                                   
2023-01-01         NaN        NaN         NaN         NaN  1569087.0   
2023-01-02         NaN        NaN         NaN         NaN  4693364.0   
2023-01-03  310.070007  89.830002  135.240005  243.080002        0.0   
2023-01-04  312.000000  91.010002  135.990005  232.279999  4618455.0   
2023-01-05  313.570007  88.070000  135.660004  227.199997  4797670.0   

Price                                                                   
Ticker             AAPL      BRK-B        GOOG         JPM        MSFT  
Date                                                                    
2023-01-01          NaN        NaN         NaN         NaN         NaN  
2023-01-02          NaN        NaN         NaN         NaN         NaN  
2023-01-03  112117500.0  3549900.0  20738500.0  11054800.0  25740000.0  
2023-01-04   89113600.0  5121200.0  27046500.0  11687600.0  50623400.0  
2023-01-05   80962700.0  3416300.0  23136100.0   8381300.0  39585600.0  

[5 rows x 36 columns]

In [85]:
ohlcv_2023_daily.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 308 entries, 2023-01-01 to 2023-12-29
Data columns (total 36 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   (Adj Close, 2222.SR)  248 non-null    float64
 1   (Adj Close, AAPL)     250 non-null    float64
 2   (Adj Close, BRK-B)    250 non-null    float64
 3   (Adj Close, GOOG)     250 non-null    float64
 4   (Adj Close, JPM)      250 non-null    float64
 5   (Adj Close, MSFT)     250 non-null    float64
 6   (Close, 2222.SR)      248 non-null    float64
 7   (Close, AAPL)         250 non-null    float64
 8   (Close, BRK-B)        250 non-null    float64
 9   (Close, GOOG)         250 non-null    float64
 10  (Close, JPM)          250 non-null    float64
 11  (Close, MSFT)         250 non-null    float64
 12  (High, 2222.SR)       248 non-null    float64
 13  (High, AAPL)          250 non-null    float64
 14  (High, BRK-B)         250 non-null    float64
 15  (Hig

In [88]:
ohlcv_2023_daily['Adj Close']

Ticker,2222.SR,AAPL,BRK-B,GOOG,JPM,MSFT
Date,,,,,,
2023-01-01,28.475794,NaN,NaN,NaN,NaN,NaN
2023-01-02,28.256075,NaN,NaN,NaN,NaN,NaN
2023-01-03,28.256075,124.216301,309.910004,89.699997,129.648499,237.036011
2023-01-04,27.860575,125.497505,314.549988,88.709999,130.857483,226.667282
2023-01-05,27.728743,124.166641,312.899994,86.769997,130.828476,219.949371
...,...,...,...,...,...,...
2023-12-25,32.927525,NaN,NaN,NaN,NaN,NaN
2023-12-26,32.877789,192.803986,356.829987,142.820007,166.387451,373.968445
2023-12-27,32.977268,192.903839,356.950012,141.440002,167.385437,373.379547


In [90]:
max_price = ohlcv_2023_daily['Adj Close'].max()
min_price = ohlcv_2023_daily['Adj Close'].min()

In [95]:
ratios = round((max_price - min_price)/max_price, 2)
ratios

Ticker
2222.SR    0.21
AAPL       0.37
BRK-B      0.21
GOOG       0.39
JPM        0.28
MSFT       0.42
dtype: float64

In [96]:
# Max Ratio is MSFT with 0.42
ratios.max()

0.42

### Question 5. \[Stocks\] Dividend Yield

[](https://github.com/DataTalksClub/stock-markets-analytics-zoomcamp/blob/main/cohorts/2024/homework%201.md#question-5-stocks-dividend-yield)

**Find the largest dividend yield for the same set of stocks**

Use the same list of companies (2222.SR,BRK-B, AAPL, MSFT, GOOG, JPM) and download all dividends paid in 2023. You can use `get_actions()` method or `.dividends` field in yfinance library ([https://github.com/ranaroussi/yfinance?tab=readme-ov-file#quick-start](https://github.com/ranaroussi/yfinance?tab=readme-ov-file#quick-start))

Sum up all dividends paid in 2023 per company and divide each value by the closing price (Adj.Close) at the last trading day of the year.

Find the maximm value in % and round to 1 digit after the decimal point. (E.g., if you obtained $1.25 dividends paid and the end year stock price is $100, the dividend yield is 1.25% -- and your answer should be equal to 1.3)

In [110]:
tickers = yf.Tickers(ticker_list)
items = []
for item in ticker_list:
    dividends  = tickers.tickers[item].dividends['2023']
    total_dividends = dividends.sum()
    hist = tickers.tickers[item].history(period="1d", start="2023-12-31", end="2024-01-01")
    if not hist.empty:
        last_price = hist.iloc[-1]['Close']
    else:
        hist = tickers.tickers[item].history(period="5d")
        last_price = hist.iloc[-1]['Close']

    if last_price > 0:
        dividend_yield = (total_dividends / last_price) * 100
        # print(round(dividend_yield, 1))
        items.append((item, round(dividend_yield, 1)))
    else:
        # print(0)
        items.append((item, 0))



BRK-B: No price data found, symbol may be delisted (1d 2023-12-31 -> 2024-01-01)
AAPL: No price data found, symbol may be delisted (1d 2023-12-31 -> 2024-01-01)
MSFT: No price data found, symbol may be delisted (1d 2023-12-31 -> 2024-01-01)
GOOG: No price data found, symbol may be delisted (1d 2023-12-31 -> 2024-01-01)
JPM: No price data found, symbol may be delisted (1d 2023-12-31 -> 2024-01-01)


In [111]:
df = pd.DataFrame(items, columns =['Ticker', 'Dividend Yield'])
df

,Ticker,Dividend Yield
0,2222.SR,2.8
1,BRK-B,0.0
2,AAPL,0.6
3,MSFT,0.7
4,GOOG,0.0
5,JPM,2.2


In [112]:
df.max()

Ticker            MSFT
Dividend Yield     2.8
dtype: object

### Question 6. \[Exploratory\] Investigate new metrics

[](https://github.com/DataTalksClub/stock-markets-analytics-zoomcamp/blob/main/cohorts/2024/homework%201.md#question-6-exploratory-investigate-new-metrics)

**Free text answer**

Download and explore a few additional metrics or time series that might be valuable for your project and write down why (briefly).

In [113]:
# Other metrics
tickers.tickers['AAPL'].financials

,2023-09-30,2022-09-30,2021-09-30,2020-09-30
Tax Effect Of Unusual Items,0.0,0.0,0.0,0.0
Tax Rate For Calcs,0.147,0.162,0.133,0.144
Normalized EBITDA,129188000000.0,133138000000.0,123136000000.0,81020000000.0
Net Income From Continuing Operation Net Minority Interest,96995000000.0,99803000000.0,94680000000.0,57411000000.0
Reconciled Depreciation,11519000000.0,11104000000.0,11284000000.0,11056000000.0
Reconciled Cost Of Revenue,214137000000.0,223546000000.0,212981000000.0,169559000000.0
EBITDA,129188000000.0,133138000000.0,123136000000.0,81020000000.0
EBIT,117669000000.0,122034000000.0,111852000000.0,69964000000.0
Net Interest Income,-183000000.0,-106000000.0,198000000.0,890000000.0
Interest Expense,3933000000.0,2931000000.0,2645000000.0,2873000000.0


In [114]:
tickers.tickers['AAPL'].info

{'address1': 'One Apple Park Way',
 'city': 'Cupertino',
 'state': 'CA',
 'zip': '95014',
 'country': 'United States',
 'phone': '408 996 1010',
 'website': 'https://www.apple.com',
 'industry': 'Consumer Electronics',
 'industryKey': 'consumer-electronics',
 'industryDisp': 'Consumer Electronics',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and pod

In [115]:
tickers.tickers['AAPL'].balance_sheet

,2023-09-30,2022-09-30,2021-09-30,2020-09-30
Treasury Shares Number,0.0,NaN,NaN,NaN
Ordinary Shares Number,15550061000.0,15943425000.0,16426786000.0,16976763000.0
Share Issued,15550061000.0,15943425000.0,16426786000.0,16976763000.0
Net Debt,81123000000.0,96423000000.0,89779000000.0,74420000000.0
Total Debt,111088000000.0,120069000000.0,124719000000.0,112436000000.0
...,...,...,...,...
Cash Cash Equivalents And Short Term Investments,61555000000.0,48304000000.0,62639000000.0,90943000000.0
Other Short Term Investments,31590000000.0,24658000000.0,27699000000.0,52927000000.0
Cash And Cash Equivalents,29965000000.0,23646000000.0,34940000000.0,38016000000.0
Cash Equivalents,1606000000.0,5100000000.0,17635000000.0,20243000000.0


Balance sheet for know the balance. Info for get information of the company like sector information, current price, etc.

### Question 7. \[Exploratory\] Time-driven strategy description around earnings releases

[](https://github.com/DataTalksClub/stock-markets-analytics-zoomcamp/blob/main/cohorts/2024/homework%201.md#question-7-exploratory-time-driven-strategy-description-around-earnings-releases)

**Free text answer**

Explore earning dates for the whole month of April - e.g. using YahooFinance earnings calendar ([https://finance.yahoo.com/calendar/earnings?from=2024-04-21&to=2024-04-27&day=2024-04-23](https://finance.yahoo.com/calendar/earnings?from=2024-04-21&to=2024-04-27&day=2024-04-23)). Compare with the previous closed earnings (e.g., recent dates with full data [https://finance.yahoo.com/calendar/earnings?from=2024-04-07&to=2024-04-13&day=2024-04-08](https://finance.yahoo.com/calendar/earnings?from=2024-04-07&to=2024-04-13&day=2024-04-08)).

Describe an analytical strategy/idea (you're not required to implement it) to select a subset companies of interest based on the future events data.

We can do the next steps:

1. Web Scraping:
    - First, we’ll scrape the relevant financial information from a website. Specifically, we’ll focus on Earnings Per Share (EPS) data.
    - We can use Python libraries like BeautifulSoup or ScraperAPI to extract this data from web pages.
2. Extract EPS Estimates by Date:
    - Once we’ve scraped the data, we’ll identify the EPS estimates for different dates. These estimates represent the expected earnings per share for specific time periods.
    - We can use methods like get_earnings_for_date in Python to retrieve this information.
3. Calculate EPS Growth:
    - To calculate EPS growth, we’ll compare the past EPS (actual earnings per share) with the future EPS (estimated earnings per share).
    - The formula for EPS growth is:
    $ \text{EPS Growth} = \frac{{\text{Future EPS} - \text{Past EPS}}}{{\text{Past EPS}}} \times 100% $
4. Machine Learning Prediction:
    - Armed with historical EPS data and growth rates, we can explore building a machine learning model.
    - ML algorithms can help predict future EPS based on various features, such as industry trends, company performance, and economic indicators.